<a href="https://colab.research.google.com/github/usseif97/MRNET-for-Knee-Diagnosis/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import applications, layers, Input
from tensorflow.keras.models import Sequential
import keras.backend.tensorflow_backend as tfback
import pandas as pd
root_path = '/content/drive/My Drive/DataSet/MRNET data set/MRNet-v1.0'



In [0]:
"""Transfer Learning"""
class Max(keras.layers.Layer):
  def __init__(self, **kwargs):
    super(Max, self).__init__(**kwargs)

  def call(self, inputs):
    self.outputs = tf.math.reduce_max(
        inputs, axis=0, keepdims=True
    )
    #self.outputs = tf.transpose(self.outputs)
    return self.outputs

  #def get_config(self):
  #  config = super(Max, self).get_config()
  #  return config


class Squeeze(keras.layers.Layer):
  def __init__(self,**kwargs):
    super(Squeeze, self).__init__(**kwargs)

  def call(self, inputs):
    self.outputs = tf.squeeze(inputs, axis = 0)
    return self.outputs

class VggModel():
  def __init__(self, root_path, view, result_label, task = 'train'):
    self.result_label = result_label
    self.root_path = root_path
    self.view = view
    self.task = task
    self.vgg_layer = applications.VGG16(
        include_top=False,
        weights="imagenet",
        input_shape=(3, 256,256),
        pooling='avg'
    )
    self.vgg_layer.trainable = False

    #vgg_layer.summary()

    inputs = keras.Input(shape=(None, 3, 256, 256))
    x = Squeeze()(inputs)
    x = applications.vgg16.preprocess_input(x)
    #inputs = keras.Input(shape=(3, 256, 256))
    x = self.vgg_layer(x, training=False)
    outputs = Max()(x)
    self.vgg_extractor = keras.Model(inputs, outputs)
    #vgg_extractor.summary()
    self.vgg_extractor.compile()

    inputs = keras.Input(shape=(512,))
    outputs = layers.Dense(1, activation='relu')(inputs)
    self.vgg_fc = keras.Model(inputs, outputs)
    #vgg_fc.summary()
    self.vgg_fc.compile(
        optimizer=keras.optimizers.Adam(1e-5), loss='binary_crossentropy', metrics=['accuracy']
        )
    
#    inputs = keras.Input(shape=(3,))
#    outputs = layers.Dense(1, activation='sigmoid')(inputs)
#    self.vgg_classifier = keras.Model(inputs, outputs)
#    #vgg_classifier.summary()
#    self.vgg_classifier.compile(
#        optimizer=keras.optimizers.Adam(1e-5), loss='binary_crossentropy', metrics=['accuracy']
#        )

  def __save_example(self, num, features):
    index = '0' * (4 - len(str(num))) + str(num)
    print('saving example:', num)
    np.save(open(f"{self.root_path}/{self.task}/{self.view}/VGG16/{index}.npy", 'wb'), features)

  def __load_labels(self):
    records = pd.read_csv(self.root_path + f"/train-{self.result_label}.csv", header=None, names=['id', 'label'])
    return records['label'].to_numpy()

  def __load_features(self):
    self.features = np.empty((0, 512))
    start = 0
    end = 0
    if self.task == 'train':
      start = 0
      end = 1130
    else:
      start = 1130
      end = 1250
    for i in range(start, end):
      index = '0' * (4 - len(str(i))) + str(i)
      path = self.root_path + f"/{self.task}/{self.view}/VGG16/{index}.npy"
      print('loading', index, 'features')
      x = np.load(path).reshape((1, 512))
      self.features = np.concatenate((self.features, x), axis = 0)


    return self.features
  
  def __save_history(self):
    with open(self.root_path + f'/models/vgg16-fc-{self.view}-{self.result_label}-history', 'wb') as file_pi:
      pickle.dump(self.history.history, file_pi)

  def plot_logs(self):
    fig = plt.figure(figsize=(20, 10))
    plt.ylim(0, 1)
    plt.plot(self.logs['accuracy'], 'g', label="train losses")
    plt.plot(self.logs['val_accuracy'], 'r', label="val losses")
    plt.grid(True)
    plt.title('Training loss vs. Validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()


  def __load_history(self):
    return pickle.load(open(self.root_path + f'/models/vgg16-fc-{self.view}-{self.result_label}-history', 'rb'))

  def load_fc(self):
    self.vgg_fc = keras.models.load_model(
      self.root_path + f'/models/vgg16-fc-{self.view}-{self.result_label}.h5', compile=True
      )
    self.train_fc_logs = self.__load_history()

  def extract_features(self, task, save = False, X = None):
    self.task = task
    self.features = np.empty((0,512))
    if X != None:
      for i in range(1130):
        print(f'extracting {i} (Memory Mode)')
        x = X[i]
        x = tf.expand_dims(x, 0)
        #x = applications.vgg16.preprocess_input(x)
        Y = self.vgg_extractor.predict(
          x, batch_size=1, verbose=0
        )
        self.features = np.concatenate((self.features,Y) , axis = 0)
    else:
      params = {'shape': (256, 256),
          'n_channels': 3,
          'batch_size': 1,
          'shuffle': False,
          'extractor': None,
          'task': self.task, 
          'view': self.view, 
          'result_label': self.result_label}

      predict_generator = MRNetDataGenerator(**params)
      self.features = self.vgg_extractor.predict(
        predict_generator, verbose=0
      )

    if save == True:
      for example_num, example_features in enumerate(self.features):
        print('saving', example_num)
        self.__save_example(example_num, example_features)

    return self.features

  def predict_fc(self):
    self.load_fc()
    X = self.__load_features()
    self.predictions = self.vgg_fc.predict(
      X, verbose=1
    )
    return self.predictions


  def train_fc(self, memory = False, save_fc = False):
    self.task = 'train'
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',
        min_delta=0,
        patience=15,
        verbose=0, mode='auto',
        restore_best_weights=True)
    
    reduce_lr = keras.callbacks.ReduceLROnPlateau(
      monitor='val_loss', factor=0.5, patience=5, verbose=0, mode='auto',
      min_delta=0, cooldown=0, min_lr=1e-7
    )
    
    if memory == True:
      Y = self.__load_labels()
      X = self.__load_features()
      #tensorboard_callback = keras.callbacks.TensorBoard(log_dir="./logs", histogram_freq=1)
      
      history = self.vgg_fc.fit(
          x=X, y=Y, batch_size=1, epochs=100, verbose=2, callbacks=[reduce_lr, early_stopping],
          validation_split=0.1, validation_data=None, shuffle=True,
      )
      self.train_fc_logs = history.history

    else:
      #Need to implement validation data
      params = {'shape': (512),
          'batch_size': 1,
          'shuffle': False,
          'extractor': 'VGG16',
          'task': self.task, 
          'view': self.view, 
          'result_label': self.result_label}
      train_generator = MRNetDataGenerator(**params)
      features[i] = self.vgg_fc.fit(
          train_generator, epochs=50, verbose=2, callbacks=[early_stopping],
          validation_split=0.0, validation_data=None, shuffle=True,
          )
    if save_fc == True:
      self.vgg_fc.save(self.root_path + f'/models/vgg16-fc-{self.view}-{self.result_label}.h5')
      __save_history()

    return self.train_fc_logs

  def train_classifier(self):
    self.task = 'train'

    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',
      min_delta=0,
      patience=15,
      verbose=0, mode='auto',
      restore_best_weights=True)
    
    Y = self.__load_labels()

    predict_fc_axial = VggModel(root_path,'axial', 'abnormal', task='train')
    predict_fc_axial.load_fc()
    predict_fc_axial.predict(
      x, batch_size=1, verbose=1
    )


    self.view = 'axial'
    X_axial = self.__load_features()
    Y_axial = self.vgg_fc.predict(
      x, batch_size=1, verbose=0
    )


    
    reduce_lr = keras.callbacks.ReduceLROnPlateau(
      monitor='val_loss', factor=0.5, patience=5, verbose=0, mode='auto',
      min_delta=0, cooldown=0, min_lr=1e-7
    )

  def validate_fc(self, memory = False):
    self.task = 'valid'
    if memory == True:
      Y = self.__load_labels()
      X = self.__load_features()
      self.vgg_fc.evaluate(
          x=X, y=X, batch_size=1, verbose=2)
    
    else:
      params = {'shape': (512),
          'batch_size': 1,
          'shuffle': False,
          'extractor': 'VGG16',
          'task': self.task, 
          'view': self.view, 
          'result_label': self.result_label}
      valid_generator = MRNetDataGenerator(**params)
      self.vgg_fc.evaluate(
          valid_generator, batch_size=1, verbose=2)
      